## Limpeza e Engenharia de Features - Clima de Brasília em 2024

### Introdução e Objetivo

A qualidade de qualquer análise de dados ou modelo de machine learning está ligada à qualidade dos dados de entrada. Dados brutos, repletos de inconsistências, valores ausentes ou formatos inadequados, podem levar a conclusões equivocadas e modelos de baixo desempenho.

O objetivo principal deste notebook é, portanto, realizar o pré-processamento completo dos dados brutos do clima de Brasília. Este processo é a fundação de todo o projeto, englobando as etapas de limpeza, tratamento de valores ausentes e, crucialmente, a engenharia de novas features (variáveis) para enriquecer o dataset.

O resultado final será um conjunto de dados limpo, robusto e estruturado, pronto para ser utilizado na fase subsequente de Análise Exploratória de Dados (EDA) e, posteriormente, na modelagem preditiva.

### Descrição do Processo

O fluxo de trabalho deste notebook está organizado nas seguintes etapas principais:

- Carga e Inspeção Inicial

- Limpeza dos Dados

- Engenharia de Features (Feature Engineering)

### Resultado Esperado

Ao final da execução deste notebook, o DataFrame processado e enriquecido será salvo em um novo arquivo. Este arquivo servirá como a fonte de dados para as análises exploratórias e modelagens subsequentes do projeto, garantindo a reprodutibilidade e a consistência do trabalho.

In [1]:
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importando os dados e inspeção inicial
df = pd.read_csv("../data/INMET_CO_DF_A001_BRASILIA_01-01-2024_A_31-12-2024.csv", encoding='windows-1252', sep=';')
print(f"Quantidade de linhas: {df.shape[0]}")
print(f"Quantidade de colunas: {df.shape[1]}")

Quantidade de linhas: 8784
Quantidade de colunas: 20


In [3]:
print(f"Resumo Geral:\n")
df.info(memory_usage='deep')

Resumo Geral:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 20 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Data                                                   8784 non-null   object 
 1   Hora UTC                                               8784 non-null   object 
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       8760 non-null   object 
 3   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  8760 non-null   object 
 4   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        8760 non-null   object 
 5   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       8760 non-null   object 
 6   RADIACAO GLOBAL (Kj/m²)                                4704 non-null   object 
 7   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           8758 non-null   object 
 8   TEMPERATURA DO PONTO DE ORVALHO (

In [4]:
df.head(2)

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2024/01/01,0000 UTC,0,"885,7","885,7","885,3",NaN,22,"18,2","22,6","21,8","18,6","18,2",81.0,77.0,79.0,311.0,"2,8","1,5",NaN
1,2024/01/01,0100 UTC,0,"886,7","886,7","885,7",NaN,"21,2","18,6",22,"21,1","18,7","18,1",85.0,79.0,85.0,315.0,"2,9","1,5",NaN


In [5]:
# Removendo coluna sem dados
df.drop(columns="Unnamed: 19", inplace=True)

### Transformando colunas de data e hora

Serão criadas novas features de datas para melhorar o modelo a encontrar padrões. Features cíclicas com `sen` e `cos` para o modelo entender que o dia 6 (Domingo) está perto do dia 0 (Segunda), ou que o mês 12 (Dezembro) está perto do mês 1 (Janeiro). A mesma regra será aplicada às horas

In [6]:
df['data'] = pd.to_datetime(df['Data'])
df.set_index('data', inplace=True)
df.sort_index(inplace=True)

In [7]:
# Criando novas features de data
df["dia_da_semana"] = df.index.dayofweek
df["dia_do_mes"] = df.index.day
df["mes"] = df.index.month
df["semana_do_ano"] = df.index.isocalendar().week

# features cíclicas
df['dia_semana_sin'] = np.sin(2 * np.pi * df['dia_da_semana']/7.0)
df['dia_semana_cos'] = np.cos(2 * np.pi * df['dia_da_semana']/7.0)

df['mes_sin'] = np.sin(2 * np.pi * df['mes']/12.0)
df['mes_cos'] = np.cos(2 * np.pi * df['mes']/12.0)

df.drop(columns=['dia_da_semana', 'mes'], inplace=True)

In [8]:
# Criando features de hora
df['hora_num'] = df['Hora UTC'].str.replace(' UTC', '').astype(int) // 100
df['hora_sin'] = np.sin(2 * np.pi * df['hora_num'] / 24)
df['hora_cos'] = np.cos(2 * np.pi * df['hora_num'] / 24)

df.drop(columns=['Data', 'Hora UTC'], inplace=True)

In [9]:
# Visualizar o resultado
df.iloc[:, -9:].sample(3, random_state=42)

,dia_do_mes,semana_do_ano,dia_semana_sin,dia_semana_cos,mes_sin,mes_cos,hora_num,hora_sin,hora_cos
data,,,,,,,,,
2024-09-23,23,39,0.000000,1.00000,-1.0,-1.836970e-16,1,0.258819,0.965926
2024-03-03,3,9,-0.781831,0.62349,1.0,6.123234e-17,0,0.000000,1.000000
2024-09-29,29,39,-0.781831,0.62349,-1.0,-1.836970e-16,11,0.258819,-0.965926


### Transformando colunas de string para float

Existem várias colunas numéricas em formatos `objects` que precisam de uma atenção. Para isso, será utilizado conversões de todas as colunas alvos para `float`

In [10]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8784 entries, 2024-01-01 to 2024-12-31
Data columns (total 26 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       8760 non-null   object 
 1   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  8760 non-null   object 
 2   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        8760 non-null   object 
 3   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       8760 non-null   object 
 4   RADIACAO GLOBAL (Kj/m²)                                4704 non-null   object 
 5   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           8758 non-null   object 
 6   TEMPERATURA DO PONTO DE ORVALHO (°C)                   8758 non-null   object 
 7   TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)             8758 non-null   object 
 8   TEMPERATURA MÍNIMA NA HORA ANT

In [11]:
columns_to_float = df.iloc[:,0:17].columns.to_list()
df[columns_to_float] = (
    df[columns_to_float]
    .replace(r'^\s*$', np.nan, regex=True)
    .replace(",", ".", regex=True)
)

# Converter para float
df[columns_to_float] = df[columns_to_float].apply(
    pd.to_numeric, errors="coerce", downcast="float"
)

### Lidando com valores ausentes

In [12]:
print(f"Quantidade de valor nulo por features:\n\n{df.isna().sum()}")

Quantidade de valor nulo por features:

PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                           24
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)      24
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)            24
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)           24
RADIACAO GLOBAL (Kj/m²)                                  4080
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)               26
TEMPERATURA DO PONTO DE ORVALHO (°C)                       26
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)                 26
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)                 26
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)           26
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)           26
UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)                   26
UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)                   27
UMIDADE RELATIVA DO AR, HORARIA (%)                        26
VENTO, DIREÇÃO HORARIA (gr) (° (gr))                       26
VENTO, RAJADA MAXIMA (m/s)    

Existem poucos valores nas colunas que irão ser removidas, com excessão à coluna de radiação. Remover metade das linhas iriam comprometer muitos dados importantes. Para lidar com os valores ausentes da coluna de radiação, será necessário uma inspeção rápida e ver como estão os valores presentes.

In [13]:
df[df["RADIACAO GLOBAL (Kj/m²)"].notna()].head()

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),...,"VENTO, VELOCIDADE HORARIA (m/s)",dia_do_mes,semana_do_ano,dia_semana_sin,dia_semana_cos,mes_sin,mes_cos,hora_num,hora_sin,hora_cos
data,,,,,,,,,,,,,,,,,,,,,
2024-01-01,0.0,885.700012,885.799988,885.500000,4.300000,19.600000,18.799999,20.100000,19.6,19.100000,...,1.1,1,1,0.0,1.0,0.5,0.866025,9,7.071068e-01,-0.707107
2024-01-01,0.0,886.400024,886.400024,885.700012,310.100006,20.600000,19.100000,20.600000,19.6,19.200001,...,1.8,1,1,0.0,1.0,0.5,0.866025,10,5.000000e-01,-0.866025
2024-01-01,0.0,886.700012,886.700012,886.400024,779.700012,21.500000,18.500000,21.600000,20.5,19.200001,...,2.1,1,1,0.0,1.0,0.5,0.866025,11,2.588190e-01,-0.965926
2024-01-01,0.0,887.200012,887.200012,886.700012,1273.500000,22.600000,18.000000,22.900000,21.5,18.600000,...,1.8,1,1,0.0,1.0,0.5,0.866025,12,1.224647e-16,-1.000000
2024-01-01,0.0,887.400024,887.500000,887.200012,1361.099976,23.299999,17.700001,23.700001,22.5,18.799999,...,2.2,1,1,0.0,1.0,0.5,0.866025,13,-2.588190e-01,-0.965926


Podemos notar que os valores presentes da Radiação Global possuem significados válidos para algumas amostras. Levando em conta que metade dos dados são valores ausente, será necessário a aplicação de métodos cautelosos para lidar com esses dados sem perder a consistência.

Deste modo, vamos interpolar os valores com a data e preencher os valores ausentes com a média mensal.

In [14]:
# Interpolação linear na coluna de radiação
df['RADIACAO GLOBAL (Kj/m²)'] = df['RADIACAO GLOBAL (Kj/m²)'].interpolate(method='linear')

# Preencher linhas com média de cada mês
df['RADIACAO GLOBAL (Kj/m²)'] = df.groupby('dia_do_mes')['RADIACAO GLOBAL (Kj/m²)'].transform(lambda x: x.fillna(x.mean()))

In [15]:
print(f"Quantidade de valor nulo por features:\n\n{df.isna().sum()}")

Quantidade de valor nulo por features:

PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                         24
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)    24
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)          24
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)         24
RADIACAO GLOBAL (Kj/m²)                                   0
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)             26
TEMPERATURA DO PONTO DE ORVALHO (°C)                     26
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)               26
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)               26
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)         26
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)         26
UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)                 26
UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)                 27
UMIDADE RELATIVA DO AR, HORARIA (%)                      26
VENTO, DIREÇÃO HORARIA (gr) (° (gr))                     26
VENTO, RAJADA MAXIMA (m/s)                               24


In [16]:
# dropando demais linhas ausentes
df.dropna(inplace=True)

### Renomeando colunas

O nome das features são extensos e informativos, mas é muito possível reduzir e manter a essência de cada coluna. Irei renomear as features que possuem essa necessidade. 

In [ ]:
df.columns

Index(['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
       'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
       'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
       'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
       'RADIACAO GLOBAL (Kj/m²)',
       'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
       'TEMPERATURA DO PONTO DE ORVALHO (°C)',
       'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
       'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
       'UMIDADE RELATIVA DO AR, HORARIA (%)',
       'VENTO, DIREÇÃO HORARIA (gr) (° (gr))', 'VENTO, RAJADA MAXIMA (m/s)',
       'VENTO, VELOCIDADE HORARIA (m/s)', 'dia_do_mes', 'semana_do_ano',
       'dia_semana_sin', 'dia_semana_cos', 'mes_sin', 'mes_cos', 'hora_num',
       'hora_sin', 'hora_cos'],
      dty

In [19]:
# Preparando dados para colunas com novos nomes
nomes_colunas = df.columns.tolist()
nomes_antigos = nomes_colunas[0:17]
nomes_novos = ["precipitacao total",
               "pressao atmos nv estacao",
               "pressao atmos max",
               "pressao atmos min",
               "radiacao",
               "temp ar",
               "temp pronto orvalho",
               "temp max",
               "temp min",
               "temp orvalho max",
               "temp orvalho min",
               "umidade max",
               "umidade min",
               "umidade relativa ar",
               "vento direcao",
               "vento rajada max",
               "vento velocidade",]

renomear_dic = dict(zip(nomes_antigos, nomes_novos))
df.rename(columns=renomear_dic, inplace=True)

Por fim, irei criar uma última feature para indicar se choveu ou não a partir de cada amostra da hora do dia.

In [22]:
df['choveu'] = (df['precipitacao total'] > 0).astype(int)
df['choveu'].value_counts()

choveu
0    8151
1     604
Name: count, dtype: int64

In [21]:
df.head(3)

,precipitacao total,pressao atmos nv estacao,pressao atmos max,pressao atmos min,radiacao,temp ar,temp pronto orvalho,temp max,temp min,temp orvalho max,...,dia_do_mes,semana_do_ano,dia_semana_sin,dia_semana_cos,mes_sin,mes_cos,hora_num,hora_sin,hora_cos,choveu
data,,,,,,,,,,,,,,,,,,,,,
2024-01-01,0.0,885.700012,885.700012,885.299988,844.651489,22.000000,18.200001,22.6,21.799999,18.600000,...,1,1,0.0,1.0,0.5,0.866025,0,0.000000,1.000000,0
2024-01-01,0.0,886.700012,886.700012,885.700012,844.651489,21.200001,18.600000,22.0,21.100000,18.700001,...,1,1,0.0,1.0,0.5,0.866025,1,0.258819,0.965926,0
2024-01-01,0.0,887.099976,887.200012,886.700012,844.651489,20.900000,18.799999,21.5,20.900000,18.900000,...,1,1,0.0,1.0,0.5,0.866025,2,0.500000,0.866025,0


### Salvando dados limpos e processados

O index é muito importante, já que foi anexado à data. Deste modo, os novos dados serão salvos em `csv` mantendo o index.

In [23]:
# inspeção final
print(f"Quantidade de dados processados: {df.shape[0]}")
print(f"Contagem final das features: {df.shape[1]}")

# salvando
diretorio_alvo = os.path.join("..\data", "INMET_DF_processado.csv")

df.to_csv(diretorio_alvo, encoding='utf-8', index=True)
print(f"Dados Processados salvo em: {diretorio_alvo}")

Quantidade de dados processados: 8755
Contagem final das features: 27
Dados Processados salvo em: ..\data\INMET_DF_processado.csv
